# 🎯 LLM-as-a-Judge

<div style="background-color: rgba(200, 200, 200, 0.1); padding: 20px; border-radius: 8px; margin-bottom: 20px; border: 1px solid rgba(127, 127, 127, 0.2); max-width: 100%; overflow-wrap: break-word;">
  <p style="font-size: 16px; line-height: 1.6">
   LLM-as-a-Judge scorers use one or more LLMs to evaluate the reliability of the original LLM's response. They offer high customizability through prompt engineering and the choice of judge LLM(s). Below is a list of the available scorers:
  </p>

*   Categorical LLM-as-a-Judge ([Manakul et al., 2023](https://arxiv.org/abs/2303.08896); [Chen & Mueller, 2023](https://arxiv.org/abs/2308.16175); [Luo et al., 2023](https://arxiv.org/abs/2303.15621))
*   Continuous LLM-as-a-Judge ([Xiong et al., 2024](https://arxiv.org/abs/2306.13063))
*   Likert Scale LLM-as-a-Judge ([Bai et al., 2023](https://arxiv.org/pdf/2306.04181))
*   Panel of LLM Judges ([Verga et al., 2024](https://arxiv.org/abs/2404.18796))
    
</div>

## 📊 What You'll Do in This Demo


<div style="display: flex; margin-bottom: 15px; align-items: center">
  <div style="background-color: #34a853; color: white; border-radius: 50%; width: 30px; height: 30px; display: flex; justify-content: center; align-items: center; margin-right: 15px; flex-shrink: 0"><strong>1</strong></div>
  <div>
    <p style="margin: 0; font-weight: bold"><a href=#section1>Set up LLM and prompts.</a></p>
    <p style="margin: 0; color: rgba(95, 99, 104, 0.8)">Set up LLM instance and load example data prompts.</p>
  </div>
</div>

<div style="display: flex; margin-bottom: 15px; align-items: center">
  <div style="background-color: #34a853; color: white; border-radius: 50%; width: 30px; height: 30px; display: flex; justify-content: center; align-items: center; margin-right: 15px; flex-shrink: 0"><strong>2</strong></div>
  <div>
    <p style="margin: 0; font-weight: bold"><a href=#section2>Generate LLM Responses and Confidence Scores</a></p>
    <p style="margin: 0; color: rgba(95, 99, 104, 0.8)">Generate and score LLM responses to the example questions using the <code>LLMPanel()</code> class.</p>
  </div>
</div>

<div style="display: flex; margin-bottom: 25px; align-items: center">
  <div style="background-color: #34a853; color: white; border-radius: 50%; width: 30px; height: 30px; display: flex; justify-content: center; align-items: center; margin-right: 15px; flex-shrink: 0"><strong>3</strong></div>
  <div>
    <p style="margin: 0; font-weight: bold"><a href=#section3>Evaluate Hallucination Detection Performance</a></p>
    <p style="margin: 0; color: rgba(95, 99, 104, 0.8)">Compute precision, recall, and F1-score of hallucination detection.</p>
  </div>
</div>

## ⚖️ Advantages & Limitations

<div style="display: flex; gap: 20px">
  <div style="flex: 1; background-color: rgba(0, 200, 0, 0.1); padding: 15px; border-radius: 8px; border: 1px solid rgba(0, 200, 0, 0.2)">
    <h3 style="color: #2e8b57; margin-top: 0">Pros</h3>
    <ul style="margin-bottom: 0">
      <li><strong>Universal Compatibility:</strong> Works with any LLM.</li>
      <li><strong>Highly Customizable:</strong> Use any LLM as a judge and tailor instruction prompts for specific use cases.</li>
    </ul>
  </div>
  
  <div style="flex: 1; background-color: rgba(200, 0, 0, 0.1); padding: 15px; border-radius: 8px; border: 1px solid rgba(200, 0, 0, 0.2)">
    <h3 style="color: #b22222; margin-top: 0">Cons</h3>
    <ul style="margin-bottom: 0">
      <li><strong>Added cost:</strong> Requires additional LLM calls for the judge LLM(s).</li>
    </ul>
  </div>
</div>

In [19]:
from uqlm import LLMPanel
from uqlm.utils import load_example_dataset

<a id='section1'></a>
## 1. Set up LLM and Prompts

In this demo, we will illustrate this approach using a set of multiple choice  questions from the [AI2-ARC benchmark](https://arxiv.org/abs/1803.05457). To implement with your use case, simply **replace the example prompts with your data**.  

In [ ]:
# Load example dataset (ai2_arc)
ai2_arc = load_example_dataset("ai2_arc", n=75)
ai2_arc.head()

Loading dataset - ai2_arc...
Processing dataset...
Dataset ready!


question answer
0  Which statement best explains why photosynthes...      A
1  Which piece of safety equipment is used to kee...      B
2  Meiosis is a type of cell division in which ge...      D
3  Which characteristic describes the texture of ...      D
4  Which best describes the structure of an atom?...      B

In [21]:
# Define prompts
MCQ_INSTRUCTION = "You will be given a multiple choice question. Return only the letter of the response with no additional text or explanation.\n"
prompts = [MCQ_INSTRUCTION + prompt for prompt in ai2_arc.question]

In this example, we use `ChatOllama` to instantiate our LLMs, but any [LangChain Chat Model](https://js.langchain.com/docs/integrations/chat/) may be used. Be sure to **replace with your LLM of choice.**

In [6]:
# import sys
# !{sys.executable} -m pip install langchain-ollama
from langchain_ollama import ChatOllama

ollama_llama = ChatOllama(model="llama2")
ollama_mistral = ChatOllama(model="mistral")
ollama_qwen = ChatOllama(model="qwen3")
# ollama_deepseek = ChatOllama(model="deepseek-r1")

In [7]:
## Alternative setup with API models

## ChatVertexAI example
# import sys
# !{sys.executable} -m pip install langchain-google-vertexai
# from langchain_google_vertexai import ChatVertexAI

# gemini_pro = ChatVertexAI(model_name="gemini-2.5-pro")
# gemini_flash = ChatVertexAI(model_name="gemini-2.5-flash")


## AzureChatOpenAI example
# import sys
# !{sys.executable} -m pip install langchain-openai

# # User to populate .env file with API credentials
# from dotenv import load_dotenv, find_dotenv
# from langchain_openai import AzureChatOpenAI

# load_dotenv(find_dotenv())
# original_llm = AzureChatOpenAI(
#     deployment_name="gpt-4o",
#     openai_api_type="azure",
#     openai_api_version="2024-02-15-preview",
#     temperature=1,  # User to set temperature
# )

<a id='section2'></a>
## 2. Generate responses and confidence scores

### `LLMPanel()` - Class for aggregating multiple instances of LLMJudge using average, min, max, or majority voting

![Sample Image](https://raw.githubusercontent.com/cvs-health/uqlm/develop/assets/images/judges_graphic.png)

#### 📋 Class Attributes

<table style="border-collapse: collapse; width: 100%; border: 1px solid rgba(127, 127, 127, 0.2);">
  <tr>
    <th style="background-color: rgba(200, 200, 200, 0.2); width: 20%; padding: 8px; text-align: left; border: 1px solid rgba(127, 127, 127, 0.2);">Parameter</th>
    <th style="background-color: rgba(200, 200, 200, 0.2); width: 25%; padding: 8px; text-align: left; border: 1px solid rgba(127, 127, 127, 0.2);">Type & Default</th>
    <th style="background-color: rgba(200, 200, 200, 0.2); width: 55%; padding: 8px; text-align: left; border: 1px solid rgba(127, 127, 127, 0.2);">Description</th>
  </tr>
  <tr>
    <td style="font-weight: bold; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">judges</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">list of LLMJudge or BaseChatModel<br><code></code></td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">Judges to use. If BaseChatModel, LLMJudge is instantiated using default parameters.</td>
  </tr>    
  <tr>
    <td style="font-weight: bold; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">llm</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">BaseChatModel<br><code>default=None</code></td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">A langchain llm `BaseChatModel`. User is responsible for specifying temperature and other relevant parameters to the constructor of the provided `llm` object.</td>
  </tr>
  <tr>
    <td style="font-weight: bold; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">system_prompt</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">str or None<br><code>default="You are a helpful assistant."</code></td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">Optional argument for user to provide custom system prompt for the LLM.</td>
  </tr>
  <tr>
    <td style="font-weight: bold; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">max_calls_per_min</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">int<br><code>default=None</code></td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">Specifies how many API calls to make per minute to avoid rate limit errors. By default, no limit is specified.</td>
  </tr>
  <tr>
    <td style="font-weight: bold; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">explanations</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">bool<br><code>default=False</code></td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">Whether to include explanations from judges alongside scores. When True, judges provide reasoning for their scores.</td>
  </tr>
  <tr>
    <td style="font-weight: bold; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">additional_context</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">str or None<br><code>default=None</code></td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">Optional argument to provide additional context to inform LLM-as-a-Judge evaluations.</td>
  </tr>

</table>

#### 🔍 Parameter Groups

<div style="display: flex; gap: 20px; margin-bottom: 20px">
  <div style="flex: 1; padding: 10px; background-color: rgba(0, 100, 200, 0.1); border-radius: 5px; border: 1px solid rgba(0, 100, 200, 0.2);">
    <p style="font-weight: bold">🧠 LLM-Specific</p>
    <ul>
      <li><code>llm</code></li>
      <li><code>system_prompt</code></li>
    </ul>
  </div>
  <div style="flex: 1; padding: 10px; background-color: rgba(0, 200, 0, 0.1); border-radius: 5px; border: 1px solid rgba(0, 200, 0, 0.2);">
    <p style="font-weight: bold">📊 Confidence Scores</p>
    <ul>
      <li><code>judges</code></li>
      <li><code>explanations</code></li>        
      <li><code>additional_context</code></li>
    </ul>
  </div>
  <div style="flex: 1; padding: 10px; background-color: rgba(200, 0, 200, 0.1); border-radius: 5px; border: 1px solid rgba(200, 0, 200, 0.2);">
    <p style="font-weight: bold">⚡ Performance</p>
    <ul>
      <li><code>max_calls_per_min</code></li>
    </ul>
  </div>
</div>

#### 💻 Usage Examples

```python
# Basic usage with single self-judge parameters
panel = LLMPanel(llm=llm, judges=[llm])

# Using two judges with default parameters
panel = LLMPanel(llm=llm, judges=[llm, llm2])

# Using two judges with default parameters
panel = LLMPanel(llm=llm, judges=[llm, llm2])

# Using judges with explanations enabled
panel_with_explanations = LLMPanel(
    llm=llm, judges=[llm, llm2], explanations=True
)
```

In [23]:
judges = [ollama_mistral, ollama_llama, ollama_qwen]
additional_context = "You are an expert in general-knowledge reasoning questions. Your task is to evaluate the correctness of the proposed answers to the provided questions."

# Option 1: With explanations
panel = LLMPanel(llm=ollama_mistral, judges=judges, additional_context=additional_context, explanations=True)

# Option 2: Without explanations
# panel = LLMPanel(llm=ollama_mistral, judges=judges, additional_context=additional_context))

### 🔄 Class Methods

<table style="border-collapse: collapse; width: 100%; border: 1px solid rgba(127, 127, 127, 0.2);">
  <tr>
    <th style="background-color: rgba(200, 200, 200, 0.2); width: 25%; padding: 8px; text-align: left; border: 1px solid rgba(127, 127, 127, 0.2);">Method</th>
    <th style="background-color: rgba(200, 200, 200, 0.2); width: 75%; padding: 8px; text-align: left; border: 1px solid rgba(127, 127, 127, 0.2);">Description & Parameters</th>
  </tr>
  <tr>
    <td style="font-weight: bold; vertical-align: top; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">LLMPanel.generate_and_score</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">
      <p>Generate responses to provided prompts and use panel to of judges to score responses for correctness.</p>
      <p><strong>Parameters:</strong></p>
      <ul>
        <li><code>prompts</code> - (<strong>list of str</strong>) A list of input prompts for the model.</li>
      </ul>
      <p><strong>Returns:</strong> <code>UQResult</code> containing data (prompts, responses, sampled responses, and confidence scores) and metadata</p>
      <div style="background-color: rgba(0, 200, 0, 0.1); padding: 8px; border-radius: 3px; margin-top: 10px; border: 1px solid rgba(0, 200, 0, 0.2); margin-right: 5px; box-sizing: border-box; width: 100%;">
        <strong>💡 Best For:</strong> Complete end-to-end uncertainty quantification when starting with prompts.
      </div>
    </td>
  </tr>
  <tr>
    <td style="font-weight: bold; vertical-align: top; padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">LLMPanel.score</td>
    <td style="padding: 8px; border: 1px solid rgba(127, 127, 127, 0.2);">
      <p>Use panel to of judges to score provided responses for correctness. Use if responses are already generated. Otherwise, use `generate_and_score`.</p>
      <p><strong>Parameters:</strong></p>
      <ul>
        <li><code>prompts</code> - (<strong>list of str</strong>) A list of input prompts for the model.</li>
        <li><code>responses</code> - (<strong>list of str</strong>) A list of LLM responses for the prompts.</li>
      </ul>
      <p><strong>Returns:</strong> <code>UQResult</code> containing data (responses and confidence scores) and metadata</p>
      <div style="background-color: rgba(0, 200, 0, 0.1); padding: 8px; border-radius: 3px; margin-top: 10px; border: 1px solid rgba(0, 200, 0, 0.2); margin-right: 5px; box-sizing: border-box; width: 100%;">
        <strong>💡 Best For:</strong> Computing uncertainty scores when responses are already generated elsewhere.
      </div>
    </td>
  </tr>
</table>

In [24]:
result = await panel.generate_and_score(prompts=prompts)

# option 2: provide pre-generated responses with score method
# result = await panel.score(prompts=prompts, responses=responses)

Output()

In [29]:
result_df = result.to_df()
result_df.head()

prompt  \
0  You will be given a multiple choice question. ...   
1  You will be given a multiple choice question. ...   
2  You will be given a multiple choice question. ...   
3  You will be given a multiple choice question. ...   
4  You will be given a multiple choice question. ...   

                                            response  judge_1  \
0   A) Sunlight is the source of energy for nearl...      1.0   
1                                  C) breathing mask      1.0   
2                                     D) ovary cells      NaN   
3                                                  D      1.0   
4   C) a network of interacting positive and nega...      0.5   

                                 judge_1_explanation  judge_2  \
0                            No explanation provided      1.0   
1                            No explanation provided      1.0   
2  The question specifically states that meiosis ...      1.0   
3                            No explanation provided      1.0   
4  While the answer "a network of interacting pos...      1.0   

                                 judge_2_explanation  judge_3  \
0  The answer correctly identifies that sunlight ...      1.0   
1  The breathing mask is used to filter air and p...      0.0   
2  Meiosis occurs in germ cells (sex cells) of or...      1.0   
3  The question asks for the texture of a kitten'...      1.0   
4  The structure of an atom is indeed a network o...      0.0   

                                 judge_3_explanation       avg  max  min  \
0  Photosynthesis is foundational because it conv...  1.000000  1.0  1.0   
1  The breathing mask (B) is designed to filter a...  0.666667  1.0  0.0   
2  Meiosis occurs in germ cells to produce haploi...       NaN  NaN  NaN   
3  The question asks about the texture of a kitte...  1.000000  1.0  1.0   
4  The correct structure of an atom is a nucleus ...  0.500000  1.0  0.0   

   median  
0     1.0  
1     1.0  
2     NaN  
3     1.0  
4     0.5

<a id='section3'></a>
## 3. Evaluate Hallucination Detection Performance

To evaluate hallucination detection performance, we 'grade' the responses against an answer key. Note the `check_letter_match` function is specific to our task (multiple choice). **If you are using your own prompts/questions, update the grading method accordingly**.

In [30]:
# Populate correct answers and grade responses
result_df["answer"] = ai2_arc.answer


def check_letter_match(response: str, answer: str):
    return response.strip().lower()[0] == answer[0].lower()


result_df["response_correct"] = [check_letter_match(r, a) for r, a in zip(result_df["response"], ai2_arc["answer"])]
result_df.head()

prompt  \
0  You will be given a multiple choice question. ...   
1  You will be given a multiple choice question. ...   
2  You will be given a multiple choice question. ...   
3  You will be given a multiple choice question. ...   
4  You will be given a multiple choice question. ...   

                                            response  judge_1  \
0   A) Sunlight is the source of energy for nearl...      1.0   
1                                  C) breathing mask      1.0   
2                                     D) ovary cells      NaN   
3                                                  D      1.0   
4   C) a network of interacting positive and nega...      0.5   

                                 judge_1_explanation  judge_2  \
0                            No explanation provided      1.0   
1                            No explanation provided      1.0   
2  The question specifically states that meiosis ...      1.0   
3                            No explanation provided      1.0   
4  While the answer "a network of interacting pos...      1.0   

                                 judge_2_explanation  judge_3  \
0  The answer correctly identifies that sunlight ...      1.0   
1  The breathing mask is used to filter air and p...      0.0   
2  Meiosis occurs in germ cells (sex cells) of or...      1.0   
3  The question asks for the texture of a kitten'...      1.0   
4  The structure of an atom is indeed a network o...      0.0   

                                 judge_3_explanation       avg  max  min  \
0  Photosynthesis is foundational because it conv...  1.000000  1.0  1.0   
1  The breathing mask (B) is designed to filter a...  0.666667  1.0  0.0   
2  Meiosis occurs in germ cells to produce haploi...       NaN  NaN  NaN   
3  The question asks about the texture of a kitte...  1.000000  1.0  1.0   
4  The correct structure of an atom is a nucleus ...  0.500000  1.0  0.0   

   median answer  response_correct  
0     1.0      A              True  
1     1.0      B             False  
2     NaN      D              True  
3     1.0      D              True  
4     0.5      B             False

In [31]:
# evaluate precision, recall, and f1-score of Semantic Entropy's predictions of correctness
from sklearn.metrics import precision_score, recall_score, f1_score

for ind in [1, 2, 3]:
    y_pred = [(s > 0) * 1 for s in result_df[f"judge_{str(ind)}"]]
    y_true = result_df.response_correct
    print(f"Judge {ind} precision: {precision_score(y_true=y_true, y_pred=y_pred)}")
    print(f"Judge {ind} recall: {recall_score(y_true=y_true, y_pred=y_pred)}")
    print(f"Judge {ind} f1-score: {f1_score(y_true=y_true, y_pred=y_pred)}")
    print(" ")

Judge 1 precision: 0.7083333333333334
Judge 1 recall: 0.6938775510204082
Judge 1 f1-score: 0.7010309278350515
 
Judge 2 precision: 0.6712328767123288
Judge 2 recall: 1.0
Judge 2 f1-score: 0.8032786885245902
 
Judge 3 precision: 0.9074074074074074
Judge 3 recall: 1.0
Judge 3 f1-score: 0.9514563106796117
 


<a id='section5'></a>
## 5. Scorer Definitions
Under the LLM-as-a-Judge approach, either the same LLM that was used for generating the original responses or a different LLM is asked to form a judgment about a pre-generated response. Below, we define two LLM-as-a-Judge scorer templates. 
### Ternary Judge Template (`true_false_uncertain`)
We follow the approach proposed by [Chen & Mueller, 2023](https://arxiv.org/abs/2308.16175) in which an LLM is instructed to score a question-response concatenation as either  *incorrect*, *uncertain*, or *correct* using a carefully constructed prompt. These categories are respectively mapped to numerical scores of 0, 0.5, and 1. We denote the LLM-as-a-judge scorers as $J: \mathcal{Y} \xrightarrow[]{} \{0, 0.5, 1\}$. Formally, we can write this scorer function as follows:

\begin{equation}
J(y_i) = \begin{cases}
    0 & \text{LLM states response is incorrect} \\
    0.5 & \text{LLM states that it is uncertain} \\
    1 & \text{LLM states response is correct}.
\end{cases}
\end{equation}

### Binary Judge Template (`true_false`)
We modify the ternary approach above and include only two categories: Correct or Incorrect, which respectively map to 1 and 0.

### Continuous Judge Template (`continuous`)
For the continuous template, the LLM is asked to directly score a question-response concatenation's correctness on a scale of 0 to 1. 

### Likert-Scale Judge Template (`likert`)
Here the judge is asked to score a question-response concatenation on a 5-point likert scale. We convert these likert scores to a [0,1] scale as follows:

\begin{equation}
J(y_i) = \begin{cases}
    0 & \text{LLM states response is completely incorrect} \\
    0.25 & \text{LLM states that it is mostly incorrect} \\
    0.5 & \text{LLM states that it is partially correct} \\
    0.75 & \text{LLM states that it is mostly correct} \\
    0.1 & \text{LLM states that it is completely correct} 
\end{cases}
\end{equation}

© 2025 CVS Health and/or one of its affiliates. All rights reserved.